In [1]:
path  = r"C:\Users\admin\Downloads\iris (1).csv"

In [2]:
import pandas as pd
import numpy as  np
import seaborn as sns
import matplotlib.pyplot as  plt


In [3]:
from   warnings  import  filterwarnings
filterwarnings('ignore')

In [4]:
df  =  pd.read_csv(path )

# basic checks and  quality checks

In [5]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [6]:
df.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [7]:
df.shape

(150, 5)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


# seperate  x  & y

In [9]:
x  =  df.drop('species'  , axis  =  1)
y  =   df['species']

# create pipeline

In [10]:
from  sklearn.pipeline import  Pipeline 
from  sklearn.preprocessing import StandardScaler 
from sklearn.impute  import SimpleImputer

In [11]:
num_pipeline =  Pipeline(steps  = [('imputer' , SimpleImputer(strategy= 'mean')) ,
                                    ('scaler' ,  StandardScaler())])

In [12]:
x_scaled  =  num_pipeline.fit_transform(x)

In [14]:
x_scaled =  pd.DataFrame(x_scaled ,  columns  = x.columns )

In [15]:
x_scaled.head()

,sepal_length,sepal_width,petal_length,petal_width
0,-0.900681,1.019004,-1.340227,-1.315444
1,-1.143017,-0.131979,-1.340227,-1.315444
2,-1.385353,0.328414,-1.397064,-1.315444
3,-1.506521,0.098217,-1.283389,-1.315444
4,-1.021849,1.249201,-1.340227,-1.315444


# train test split

In [16]:
from  sklearn.model_selection import train_test_split

In [17]:
x_train  , x_test  , y_train ,  y_test  =  train_test_split(x_scaled  ,  y , random_state= 2  ,  test_size  =  0.15  )

In [18]:
print(x_train.shape  ,  y_train.shape)
print(x_test.shape  , y_test.shape)

(127, 4) (127,)
(23, 4) (23,)


# model  fitting

In [ ]:
from  sklearn.tree  import  DecisionTreeClassifier
model  =   DecisionTreeClassifier(max_depth= 10 ,  min_samples_split= 10  , )

In [28]:
model.fit(x_train  , y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,10
,min_samples_split,10
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [29]:
model.score(x_train  , y_train)

0.984251968503937

In [30]:
model.score(x_test  , y_test)

0.9565217391304348

# model eveluation

In [31]:
from sklearn.metrics  import  classification_report ,  confusion_matrix

In [33]:
# training evaluation
y_train_pred  =  model.predict(x_train)
print(confusion_matrix( y_train  ,  y_train_pred))
print(classification_report(y_train ,  y_train_pred))

[[39  0  0]
 [ 0 42  2]
 [ 0  0 44]]
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        39
  versicolor       1.00      0.95      0.98        44
   virginica       0.96      1.00      0.98        44

    accuracy                           0.98       127
   macro avg       0.99      0.98      0.98       127
weighted avg       0.98      0.98      0.98       127



# testing evaluation

In [35]:
y_test_pred   =  model.predict(x_test)
print(confusion_matrix(y_test  ,  y_test_pred))
print(classification_report(y_test  ,  y_test_pred))

[[11  0  0]
 [ 0  5  1]
 [ 0  0  6]]
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        11
  versicolor       1.00      0.83      0.91         6
   virginica       0.86      1.00      0.92         6

    accuracy                           0.96        23
   macro avg       0.95      0.94      0.94        23
weighted avg       0.96      0.96      0.96        23



# Hyperparameter tunning
#### Grid  search cv

In [42]:
from sklearn.model_selection import  GridSearchCV
from  sklearn.tree  import  DecisionTreeClassifier
DT  = DecisionTreeClassifier()

In [56]:
para =  {"max_depth" : range(2 , 15) ,  
           "criterion"  :  ['gini' ,  'entropy'] }
gscv   =  GridSearchCV(DT ,  cv  =  5  , param_grid =  para)


In [57]:
gscv.fit(x_train  ,  y_train )

,estimator,DecisionTreeClassifier()
,param_grid,"{'criterion': ['gini', 'entropy'], 'max_depth': range(2, 15)}"
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'gini'


In [58]:
gscv.best_params_

{'criterion': 'gini', 'max_depth': 4}

In [59]:
gscv.best_score_

np.float64(0.9763076923076923)

In [60]:
gscv.cv

5

In [62]:
gscv.score(x_train  , y_train)

1.0

In [63]:
gscv.score(x_test  , y_test)

0.9130434782608695

In [64]:
gscv_y_train_pred = gscv.predict(x_train)
gscv_y_test_pred = gscv.predict(x_test)

# Training evaluation


In [65]:
print(confusion_matrix(y_train ,  gscv_y_train_pred))
print(classification_report(y_train  , gscv_y_train_pred))

[[39  0  0]
 [ 0 44  0]
 [ 0  0 44]]
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        39
  versicolor       1.00      1.00      1.00        44
   virginica       1.00      1.00      1.00        44

    accuracy                           1.00       127
   macro avg       1.00      1.00      1.00       127
weighted avg       1.00      1.00      1.00       127



# testing  evaluation

In [66]:
print(confusion_matrix(y_test ,  gscv_y_test_pred))
print(classification_report(y_test  , gscv_y_test_pred))

[[11  0  0]
 [ 0  5  1]
 [ 0  1  5]]
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        11
  versicolor       0.83      0.83      0.83         6
   virginica       0.83      0.83      0.83         6

    accuracy                           0.91        23
   macro avg       0.89      0.89      0.89        23
weighted avg       0.91      0.91      0.91        23



In [67]:
x_sample  = pd.read_csv(r"C:\Users\admin\Downloads\iris_sample.csv")

In [68]:
x_sample_ = num_pipeline.transform(x_sample)

In [69]:
x_sample_ =  pd.DataFrame(x_sample_ ,  columns  = x.columns)

In [70]:
output_model  =  pd.DataFrame()

In [71]:
model_pred  =   model.predict(x_sample_)

In [73]:
output_model['model_pred'] =  model_pred

In [74]:
output_model

,model_pred
0,versicolor
1,virginica
2,versicolor
3,setosa
4,virginica
5,setosa
6,versicolor
7,virginica
8,setosa
9,versicolor


In [75]:
gscv_pred =  gscv.predict(x_sample_)

In [76]:
output_model['gscv_pred'] =  gscv_pred

In [77]:
output_model

,model_pred,gscv_pred
0,versicolor,versicolor
1,virginica,virginica
2,versicolor,versicolor
3,setosa,setosa
4,virginica,virginica
5,setosa,setosa
6,versicolor,versicolor
7,virginica,virginica
8,setosa,setosa
9,versicolor,versicolor
